# Bioinformatics project - part 1


In this part we designed and wrote a program the does the following:

<br>

Given a set of COG-spelled genomes S, where each genome is segmented into segments such that each segment could contain one or more operons, and given parameters Q, l and an unknown COG X, finds all strings c of length l that are conserved in at least q of the genoomes in the database S, such the our unknown COg appears at least once in c.

<br>

The COG chosen by us is COG 0673.



#### Let's start!

## 1. Initializing parameters and imports

In [44]:
import os
import pandas as pd

In [45]:
cog = '0673'
q = 5
l = 5 # we tool all strings c where  2 <= |c| <= 10 to start with and to get more information for the biological analysis */


## 2. Searching for all words with our cog

In [46]:
def search_words(database, cog):
    lst = []
    for index, row in database.iterrows():
        if cog in row['E']:
            lst.append([row['E'], row['B']])
    return lst

 > Search_words goes through every single line on our database and gathers all instances where our cog is in. <br>
 We also record each organism where the line was presented for later use.

In [47]:
words_bact = pd.read_csv(".\cog_words_bac.csv") #loading the first database
words_plasm = pd.read_csv(".\cog_words_plasmid.csv") #loading the second database

bact_lst = search_words(words_bact, cog)
plasm_lst = search_words(words_plasm, cog)
full_lst = bact_lst + plasm_lst

This is the "heaviest" function in our program, since it is looping over the entire database and looking for instances of our cog. <br>
The time complexity is $O(|DATABASE|*|COG|)$, and since |cog|=4 which is constant the time complexity overall is $O(|DATABASE|)$

## 2.1 Encoding our list

Our list is now made of 2-items lists: in the first item(<full_lst[i][0]>) there is a word which contains our COG, and the second item(<full_lst[i][1]>) contatins the organism that had this word in it's genome. 

In [48]:
print(full_lst[8])
print(full_lst[8][0])
print(full_lst[8][1])

['505\t0079\t0673\t3475\t1211\t0451\tX\tX\t0463\t2244\t1209\t1088\t1091\t0463\t1898\t1088\t', 'NC_016077']
505	0079	0673	3475	1211	0451	X	X	0463	2244	1209	1088	1091	0463	1898	1088	
NC_016077


As you can see, our words are made out of 3-4 digits "letters" which are seperated by tabs. on the next stage we want to extract all substrings of each words

- Due to the sturcture of the words, it may be inconvenient to loop through the words and letters, so we decided to enconde each letter(which is actually an integer) into it's ascii character. <br>
- There are over 10,000 characters which are encoded in ascii, so each COG will be mapped to exactly one character in an injective way. <br>
- That way we know we won't harm our words, and after removing the tabs we will get words just like we know them, which are easy to work with.

In [49]:
def encode(words):
    coded_lst = []
    for word in words:
        organism = word[1]
        word = word[0]
        index = 0
        right = 0
        str = ''
        while right != -1:
            if word[index] == 'X':
                str += 'X'
                right = get_next(word, right)
                index = right
                continue
            right = get_next(word, right)
            ascii = int(word[index:right - 1])
            index = right
            char = chr(ascii)
            str += char
        coded_lst.append([str, organism])
    return coded_lst

def get_next(str, index):
    len1 = len(str)
    while str[index] != '\t':
        index += 1
    index += 1
    return index if index < len1 else -1

 > The words might not be distinguishable to human eyes, but the computer can tell the differences due to the way Strings are stored in memory

In [50]:
coded_lst = encode(full_lst)
print(coded_lst[8])

['ǹOʡඓһǃXXǏ\u08c4ҹруǏݪl', 'NC_016077']


## 2.2 Getting all substrings with our COG

Now that our list is ready to work with, we want to get all substrings in length K which contains our cog. <br>

- *we won't forget to count each appearance of each substring, since multiple appearances might tell us something about this particular string.* <br>

- *we also won't forget to keep the organism in which each substring was in, for the next section.*

In [52]:
def get_subs(str, counter, length, organism):
    subs = []
    for i in range(len(str)):
        sub = str[i: i + length]
        if len(sub) == length and chr(int(cog)) in sub:
            if sub in counter:
                counter[sub][0] += 1
                counter[sub][2].add(organism)
            else:
                counter[sub] = [1, len(sub), set()]
                counter[sub][2].add(organism)

This is a simple program to find substrings. For a String with size s, we want to generate all substrings with length k which contains our COG.<br>

This is being done in $O(k*|S|+|S|) = O(k*|S|)$ for each string.

In [60]:
counter = {}
for j in range(len(coded_lst)): #loop over every string in our list
    for i in range(2, 11): #get all strings with lengths 2 to 10
        get_subs(coded_lst[j][0], counter, i, coded_lst[j][1])

> From this point onwards, we will refer to our coded_lst size as "n". <br>

>we won't forget that n <<< |DATABASE|

That's why the process of getting all subs is **************TODO - explain about time complexity and dictionary count************

### 2.3 Filtering according to q parameter

Now that we have all substrings kept in a dictionary, it's time to keep only the substrings which appeared on at least q different organisms.

if we take a look on our dictionary, we can see it's made of keys, which are our substrings, and values, which are now 3-items lists:
- counter['sub'][0] contains the number of times we witnessed this particular sub. <br>

- counter['sub'][1] contains the length of the substring

- counter['sub'][2] contains a set of all the genomes where this substring appeared.

In [61]:
print(counter['ॢʡÑ\x14'])

[1, 4, {'NC_013209'}]


> As you can see, despite the fact this substring seems to in length 7, it is actually has only 4 cogs in it. 


> We can also see it appears in only one COG. since our q value is 5, we want to filter it out.

In [62]:
counter = {k: v for k, v in counter.items() if len(v[2]) > q} # filtering according to q value

In [63]:
print('ॢʡÑ\x14' in counter)

False


#### Now that we have filtered out the unnecessary strings, we can decode our strings again!

In [64]:
def decode(counter):
    dict = {}
    for key in counter.keys():
        res = ''
        for char in key:
            code = str(ord(char))
            code = (4 - len(code))*'0'+code
            res += code
            res += '\t'
        dict[res] = counter[key]
    return dict

In [65]:
counter = decode(counter)

In [66]:
#uncomment and run the next line of code in order to filter and get only subs with length l
#counter = {k: v for k, v in counter.items() if v[1] == l}
#however, we won't do this for now, since we want all substrings with length 2 to 10 for a more complete biological analysis

### 2.4 Issuing a csv report with all the information

In [67]:
df = pd.DataFrame.from_dict(counter, orient='index').reset_index()
df = df.rename(columns={'index': 'word', 1: 'length', 0: 'occurrences', 2: 'organisms'})
s = 'report_' + cog + '.csv'
df.to_csv(f"./{s}", index=False)

Now you can see a csv report with all the information is available in the directory.

> For better presentation, it is recommended to download the csv file and open it on your computer

### Bonus stage: analysing the csv file for a better representation of our results



Now that we have all our relevant data ready, we want to present each word made of cogs and align it with their known functionality

In [88]:
def analyze(cog):
    df = pd.read_csv(f"./report_{cog}.csv")
    new_df = df[df['length'] >= 5] #we want to take only long strings with our cog
    results = []
    for index, row in new_df.iterrows(): #splitting the tabs from our strings
        cogs = row['word'][:-1]
        cog_list = cogs.split('\t')
        results.append([cog_list, row['occurrences']])
        
    cog_table = pd.read_csv("./cog_info_table.csv") #this table contains the functionality of each cog, so we are loading it for now.

    to_remove = []
    for result in results: #now we want to remove all sequences which are substrings of other sequences. This is for better visualisation
        for result2 in results:
            if all(elem in result[0] for elem in result2[0]) and result[0] != result2[0] and result[0] not in to_remove:
                to_remove.append(result2[0])
                break

    results = [x for x in results if x[0] not in to_remove]
    with open(f"./final_report_{cog}.txt", "w") as report:
        for lst in results:
            lst_to_str = '\t'.join(lst[0]) + '\t'
            ls = str(lst[0]) + f"    ||||||||     num of occurances: {lst[1]}\n"
            flag = False
            for cog2 in lst[0]:
                d = cog_table.loc[cog_table['A'] == 'COG{}'.format(cog2), 'D'].values
                e = cog_table.loc[cog_table['A'] == 'COG{}'.format(cog2), 'E'].values
                s = d + '|' + e
                try:
                    if 'Translation' in s[0] or 'translation' in s[0]:
                        i = 0
                        flag = True
                    else:
                        ls = ls + s + '\n'
                except IndexError:
                    print(f'cog_{cog2} is faulty')
                    ls = ls + f'Unknown_{cog2}\n'
            if not flag:
                report.write(''.join(ls))
    report.close()

In [89]:
analyze(cog)

## Now you can open "final_report_{cog}" in the directory and start your biological analysis!